In [2]:
from sql_connector import mydb, mycursor
import numpy as np
import pandas as pd
import random
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Download necessary NLTK data
nltk.download('punkt', download_dir=r'C:\Users\pelegel\PycharmProjects\pythonProject1\nltk')
nltk.download('stopwords', download_dir=r'C:\Users\pelegel\PycharmProjects\pythonProject1\nltk')
nltk.data.path.append(r'C:\Users\pelegel\PycharmProjects\pythonProject1\nltk')

import nltk
import ssl


# Set seed for reproducibility
np.random.seed(42)
random.seed(42)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pelegel\PycharmProjects\pythonProject1\nltk..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pelegel\PycharmProjects\pythonProject1\nltk..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Extract columns names
mycursor.execute("SHOW COLUMNS FROM en_tweets")
tweet_columns = [f"t.{row[0]}" for row in mycursor.fetchall()]
mycursor.execute("SHOW COLUMNS FROM en_users")
user_columns = [f"u.{row[0]}" for row in mycursor.fetchall()]
mycursor.execute("SHOW COLUMNS FROM en_retweets")
retweet_columns = [f"r.{row[0]}" for row in mycursor.fetchall()]

# Extract tweets with retweets -> Class 1
query = f"""
    SELECT {', '.join(tweet_columns + user_columns)}
    FROM en_retweets r
    JOIN en_users u ON r.original_author_id = u.id
    JOIN en_tweets t ON r.original_tweet_id = t.id
    WHERE t.created_at <= '2022-12-30 17:00:00'
    GROUP BY t.id;
"""
mycursor.execute(query)
tweets_t = mycursor.fetchall()

# Combine the column names for the DataFrame
column_names = tweet_columns + user_columns
tweets_t = pd.DataFrame(tweets_t, columns=column_names)
print(tweets_t.shape)

# Extract all tweets
query = f"""
    SELECT {', '.join(tweet_columns + user_columns)}
    FROM en_tweets t
    JOIN en_users u ON t.author_id = u.id
    WHERE t.created_at <= '2022-12-30 17:00:00';
"""
mycursor.execute(query)
tweets_all = mycursor.fetchall()
column_names = tweet_columns + user_columns
tweets_all = pd.DataFrame(tweets_all, columns=column_names)
print(tweets_all.shape)

# Extract original tweets that got retweeted
mycursor.execute("""SELECT original_tweet_id FROM en_retweets WHERE original_tweet_id IS NOT NULL;""")
original_tweets = mycursor.fetchall()
column_names = [desc[0] for desc in mycursor.description]
original_tweets = pd.DataFrame(original_tweets, columns=column_names)
original_tweet_id = set(original_tweets['original_tweet_id'])

# Extract tweets with no retweets -> Class 0
tweets_no_retweeets = list(set(tweets_all['t.id']) - set(original_tweets['original_tweet_id']))
tweets_f = pd.DataFrame(tweets_all[tweets_all['t.id'].isin(tweets_no_retweeets)]) # tweets with no retweets
tweets_f.loc[:, 'label'] = 0
tweets_t.loc[:, 'label'] = 1

print("tweets class 1:", tweets_t.shape)
print("tweets class 0:", tweets_f.shape)

X_t = pd.concat([tweets_t, tweets_f], axis=0)
y_t = X_t['label']


(15009, 256)
(321061, 256)
tweets class 1: (15009, 257)
tweets class 0: (306052, 257)


In [4]:
# Add week column
from datetime import datetime

X_t['t.created_at'] = pd.to_datetime(X_t['t.created_at'])

def assign_week(date):
    if date <= datetime(2022, 12, 9, 17, 0, 0) :
        return 1
    elif datetime(2022, 12, 9, 17, 0, 0)  <= date <= datetime(2022, 12, 16, 17, 0, 0) :
        return 2
    elif datetime(2022, 12, 16, 17, 0, 0)  <= date <= datetime(2022, 12, 23, 17, 0, 0) :
        return 3
    elif datetime(2022, 12, 2, 17, 0, 0)  <= date <= datetime(2022, 12, 30, 17, 0, 0) :
        return 4
    else:
        print(date)

# Apply the manual week mappings
X_t['week'] = X_t['t.created_at'].apply(assign_week)
print(X_t['week'])


0         1
1         1
2         1
3         1
4         1
         ..
321056    4
321057    4
321058    4
321059    4
321060    4
Name: week, Length: 321061, dtype: int64


In [5]:
# Preprocessing function
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|\#\w+', '', text)
    # Remove special characters and digits
    text = re.sub(r'\W|\d', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize (simple split)
    tokens = text.split()
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Remove stopwords and specific words, then lemmatize
    stop_words = set(stopwords.words('english') + ['chatgpt', 'gpt', 'chat gpt', 'chat'])
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return ' '.join(tokens)

# Apply preprocessing to the tweets
tweets_all['processed_text'] = tweets_all['t.text'].apply(preprocess_text)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000, max_df=0.95, min_df=2)
tfidf = vectorizer.fit_transform(tweets_all['processed_text'])



In [6]:
import re
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import NMF
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import numpy as np

# Function to get top words for each topic
def get_top_words_for_nmf_components(nmf_model, feature_names, num_words=10):
    word_dict = {}
    for i, topic_vector in enumerate(nmf_model.components_):
        top_words_ids = topic_vector.argsort()[-num_words:][::-1]
        top_words = [feature_names[id] for id in top_words_ids]
        word_dict[f'Topic {i+1}'] = top_words
    return word_dict

# Function to compute coherence score
def compute_coherence_score(nmf_model, tfidf, vectorizer, tweets_all):
    feature_names = vectorizer.get_feature_names_out()
    word_dict = get_top_words_for_nmf_components(nmf_model, feature_names)
    
    # Prepare documents in a format suitable for Gensim
    documents = [[word for word in doc.split() if word in feature_names] for doc in tweets_all['processed_text']]
    
    # Create a dictionary and corpus for Gensim
    gensim_dictionary = Dictionary(documents)
    gensim_corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]
    
    # Train a coherence model
    coherence_model = CoherenceModel(topics=[list(word_dict[f'Topic {i+1}']) for i in range(nmf_model.n_components)],
                                     texts=documents,
                                     dictionary=gensim_dictionary,
                                     coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    return coherence_score

# Determine optimal number of topics
num_topics_range = range(5, 11, 1)  # 5-10 topics
coherence_scores = {}

for num_topics in num_topics_range:
    nmf_model = NMF(n_components=num_topics, random_state=42)
    nmf_output = nmf_model.fit_transform(tfidf)
    coherence_score = compute_coherence_score(nmf_model, tfidf, vectorizer, tweets_all)
    coherence_scores[num_topics] = coherence_score
    print(f'Number of Topics: {num_topics}, Coherence Score: {coherence_score}')

# Identify the best number of topics
best_num_topics = max(coherence_scores, key=coherence_scores.get)
print(f'Optimal Number of Topics: {best_num_topics}')

# Train NMF with the best number of topics
nmf_model = NMF(n_components=best_num_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf)

# Get the top words for the best topics
feature_names = vectorizer.get_feature_names_out()
word_dict = get_top_words_for_nmf_components(nmf_model, feature_names)

# Print the topics and their top words
print("\nTop Words for Each Topic:")
for topic, words in word_dict.items():
    print(f"{topic}: {', '.join(words)}")

Number of Topics: 5, Coherence Score: 0.5698422065926902
Number of Topics: 6, Coherence Score: 0.5616397673309983
Number of Topics: 7, Coherence Score: 0.5655517401811604
Number of Topics: 8, Coherence Score: 0.5714632351486831
Number of Topics: 9, Coherence Score: 0.5728598837312471
Number of Topics: 10, Coherence Score: 0.5689931633456018
Optimal Number of Topics: 9

Top Words for Each Topic:
Topic 1: like, think, good, get, make, thing, one, would, time, human
Topic 2: google, search, code, red, engine, replace, business, better, result, threat
Topic 3: write, asked, poem, code, story, song, style, tweet, essay, result
Topic 4: ask, question, let, code, want, someone, tell, anything, maybe, write
Topic 5: use, case, way, still, tool, create, person, via, content, free
Topic 6: new, via, openai, chatbot, bot, artificial, intelligence, world, article, amp
Topic 7: answer, question, give, asked, asking, wrong, stack, correct, overflow, right
Topic 8: know, need, even, everything, thing

In [7]:
# Add topic modeling
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')
nltk.download('punkt_tab', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')

def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'@\w+|\#\w+', '', text)  # Remove mentions and hashtags
    text = re.sub(r'\W|\d', ' ', text)  # Remove special characters and digits
    text = text.lower()    # Convert to lowercase
    tokens = word_tokenize(text)   # Tokenize
    lemmatizer = WordNetLemmatizer()    # Initialize lemmatizer
    # Remove stopwords and specific words, then lemmatize
    stop_words = set(stopwords.words('english') + ['chatgpt', 'gpt', 'chat gpt', 'chat'])
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return ' '.join(tokens)

# Apply preprocessing to the tweets
X_t['processed_text'] = X_t['t.text'].apply(preprocess_text)

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000, max_df=0.95, min_df=2)
tfidf = vectorizer.fit_transform(X_t['processed_text'])

# Apply NMF
num_topics = best_num_topics
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf)

# Get the top words for each topic
feature_names = vectorizer.get_feature_names_out()
word_dict = {}
for i in range(num_topics):
    words_ids = nmf_model.components_[i].argsort()[:-10 - 1:-1]
    words = [feature_names[key] for key in words_ids]
    word_dict[f'Topic {i+1}'] = words

# Print the topics and their top words
for topic, words in word_dict.items():
    print(f"{topic}: {', '.join(words)}")

# Assign topics to tweets
X_t['dominant_topic'] = nmf_output.argmax(axis=1)

# Print a few examples
print("\nExample tweets and their dominant topics:")
for i in range(5):  # Print first 5 examples
    tweet = X_t['t.text'].iloc[i]
    topic = X_t['dominant_topic'].iloc[i]
    print(f"Tweet: {tweet}")
    print(f"Dominant Topic: Topic {topic + 1}")
    print(f"Top words: {', '.join(word_dict[f'Topic {topic + 1}'][:5])}\n")

print("All Distinct Topics:")
for topic, words in word_dict.items():
    print(f"{topic}: {', '.join(words)}")

print(X_t['dominant_topic'].value_counts())
X_t['dominant_topic'] = X_t['dominant_topic'] + 1
one_hot_encoded = pd.get_dummies(X_t['dominant_topic'], prefix='topic')
X_t = pd.concat([X_t, one_hot_encoded], axis=1)

[nltk_data] Downloading package punkt to C:\\Users\\pelegel\\PycharmPr
[nltk_data]     ojects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\\Users\\pelegel\\Pycha
[nltk_data]     rmProjects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Topic 1: like, think, good, get, make, thing, one, would, time, human
Topic 2: google, search, code, red, engine, replace, business, better, result, threat
Topic 3: write, asked, poem, code, story, song, style, tweet, essay, got
Topic 4: ask, question, let, code, want, someone, tell, anything, maybe, get
Topic 5: use, case, way, still, tool, create, person, via, content, free
Topic 6: new, via, openai, chatbot, bot, artificial, intelligence, world, article, amp
Topic 7: answer, question, give, asked, asking, wrong, stack, correct, overflow, right
Topic 8: know, need, even, everything, thing, let, want, well, openai, right
Topic 9: using, code, generated, create, generate, help, content, work, start, thread

Example tweets and their dominant topics:
Tweet: I asked ChatGPT what it thinks about Mutiny Web https://t.co/dOi1pFNQ16
Dominant Topic: Topic 3
Top words: write, asked, poem, code, story

Tweet: People tricking ChatGPT “like watching an Asimov novel come to life” https://t.co/DKc4C

In [8]:
# Remove retweets columns
X_t = X_t.drop(columns=[col for col in X_t.columns if col.startswith('r.')])
X_t = X_t.drop(columns=['t.id', 't.edit_history_tweet_ids', 't.created_at', 't.lang', 't.text', 't.author_id', 't.screen_name',
                        't.entities', 't.geo', 't.withheld', 't.created_at', 't.in_reply_to_user_id', 't.retweet_count', 't.reply_count',
                        't.like_count', 't.quote_count', 't.impression_count', 't.original_author_screen_name',
                        'u.id', 'u.id_str', 'u.name', 'u.screen_name', 'u.location', 'u.profile_location', 'u.description', 'u.entities',
                        'u.created_at', 'u.utc_offset', 'u.time_zone', 'u.lang', 'u.status', 'u.translator_type',
                        'u.withheld_in_countries',
                        'u.a1_in_degree', 'u.b1_in_degree', 'u.c1_in_degree',
                        'u.a2_in_degree', 'u.b2_in_degree', 'u.c2_in_degree',
                        'u.a3_in_degree', 'u.b3_in_degree', 'u.c3_in_degree',
                        'u.a4_in_degree', 'u.b4_in_degree', 'u.c4_in_degree',
                        'u.a1_closeness', 'u.b1_closeness', 'u.c1_closeness',
                        'u.a2_closeness', 'u.b2_closeness', 'u.c2_closeness',
                        'u.a3_closeness', 'u.b3_closeness', 'u.c3_closeness',
                        'u.a4_closeness', 'u.b4_closeness', 'u.c4_closeness', 't.week', 'processed_text', 'dominant_topic',
                        'label'])

X_t.to_parquet('X_t_tm_new.parquet', index=False)
y_t_df = pd.DataFrame(y_t)
y_t_df.to_parquet('y_t_tm_new.parquet', index=False)

print(X_t.columns.to_list())

correlations = X_t.corrwith(y_t)
correlations = correlations.abs().sort_values(ascending=False)
print(correlations.head(30))

['t.hashtags', 't.urls', 't.user_mentions', 't.admiration', 't.amusement', 't.anger', 't.annoyance', 't.approval', 't.caring', 't.confusion', 't.curiosity', 't.desire', 't.disappointment', 't.disapproval', 't.disgust', 't.embarrassment', 't.excitement', 't.fear', 't.gratitude', 't.grief', 't.joy', 't.love', 't.nervousness', 't.neutral', 't.optimism', 't.pride', 't.realization', 't.relief', 't.remorse', 't.sadness', 't.surprise', 't.seniority_at_tweeting', 'u.protected', 'u.followers_count', 'u.friends_count', 'u.listed_count', 'u.favourites_count', 'u.geo_enabled', 'u.verified', 'u.statuses_count', 'u.contributors_enabled', 'u.is_translator', 'u.is_translation_enabled', 'u.has_extended_profile', 'u.default_profile', 'u.default_profile_image', 'u.following', 'u.follow_request_sent', 'u.notifications', 'u.retweets_num', 'u.admiration_avg', 'u.admiration_min', 'u.admiration_max', 'u.admiration_std', 'u.amusement_avg', 'u.amusement_min', 'u.amusement_max', 'u.amusement_std', 'u.anger_avg',

C:\Users\pelegel\PycharmProjects\pythonProject1\myenv\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\pelegel\PycharmProjects\pythonProject1\myenv\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


u.listed_count              0.073145
t.user_mentions             0.062612
u.favourites_count          0.051370
t.urls                      0.049193
u.default_profile           0.040981
u.followers_count           0.040612
t.seniority_at_tweeting     0.038330
u.seniority                 0.038330
u.friends_count             0.036982
u.a3_betweenness            0.036166
u.geo_enabled               0.032121
u.statuses_count            0.031495
u.a1_betweenness            0.028137
t.approval                  0.028038
t.neutral                   0.025638
t.realization               0.023934
u.curiosity_avg             0.022429
u.confusion_avg             0.021222
u.curiosity_std             0.021106
week                        0.020647
t.optimism                  0.019993
u.confusion_std             0.019686
u.a4_out_degree             0.018608
u.realization_min           0.018544
u.curiosity_max             0.018475
u.is_translation_enabled    0.018379
u.b1_betweenness            0.018291
u